In [1]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.7.2"
JULIA_PACKAGES="PyCall SymbolicRegression"
JULIA_NUM_THREADS=4
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"'
  done
  
  julia -e 'println("Success")'
fi

Output streaming troncato alle ultime 5000 righe.
  ◒ RecursiveArrayTools
  ✓ ArrayInterfaceStaticArrays
  ◒ FiniteDiff
  Progress [==============================>          ]  82/110
  ◐ BangBang
  ◐ RecursiveArrayTools
  ✓ ArrayInterfaceStaticArrays
  ◐ FiniteDiff
  Progress [==============================>          ]  82/110
  ◓ BangBang
  ◓ RecursiveArrayTools
  ✓ ArrayInterfaceStaticArrays
  ◓ FiniteDiff
  Progress [==============================>          ]  82/110
  ◑ BangBang
  ◑ RecursiveArrayTools
  ✓ ArrayInterfaceStaticArrays
  ◑ FiniteDiff
  Progress [==============================>          ]  82/110
  ◒ BangBang
  ◒ RecursiveArrayTools
  ✓ ArrayInterfaceStaticArrays
  ◒ FiniteDiff
  Progress [==============================>          ]  82/110
  ◐ BangBang
  ◐ RecursiveArrayTools
  ✓ ArrayInterfaceStaticArrays
  ◐ FiniteDiff
  Progress [==============================>          ]  82/110
  ◓ BangBang
  ◓ RecursiveArrayTools
  ✓ ArrayInterfaceStaticArrays
  ◓ FiniteDiff
  Pr

In [ ]:
#%pip install -Uq pysr pytorch_lightning

In [ ]:
'''
from julia import Julia

julia = Julia(compiled_modules=False)
from julia import Main
from julia.tools import redirect_output_streams

redirect_output_streams()


In [2]:
import pickle as pkl

In [3]:
#accedi alla cartella di Google Drive Colab Notebook, dove è presente il file messages_over_time.pkl generato dal codice GNN  

from google.colab import drive
drive.mount("/content/drive")

%cd /content/drive/MyDrive/Colab Notebooks
#%cd /content/drive/MyDrive/GNN_project
%ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks
FvsMessages.mp4
GN_Demo_Colab.ipynb
GN_Demo_Colab_Vincenzo.ipynb
hall_of_fame_2022-11-03_141336.720.csv
hall_of_fame_2022-11-03_141336.720.csv.bkup
hall_of_fame_2022-11-03_141336.720.pkl
hall_of_fame_2022-11-04_083749.351.csv
hall_of_fame_2022-11-04_083749.351.csv.bkup
hall_of_fame_2022-11-04_083749.351.pkl
hall_of_fame_2022-11-04_091308.511.csv
hall_of_fame_2022-11-04_091308.511.csv.bkup
hall_of_fame_2022-11-04_091308.511.pkl
hall_of_fame_2022-11-05_112040.351.csv
hall_of_fame_2022-11-05_112040.351.csv.bkup
hall_of_fame_2022-11-05_112040.351.pkl
hall_of_fame_2022-11-05_114009.257.csv
hall_of_fame_2022-11-05_114009.257.csv.bkup
hall_of_fame_2022-11-05_114009.257.pkl
hall_of_fame_2022-11-05_115747.529.csv
hall_of_fame_2022-11-05_115747.529.csv.bkup
hall_of_fame_2022-11-05_115747.529.pkl
_init_.py
messages_over_time.pkl
mo

In [4]:
#scarica e carica il file .pkl da utilizzare per la regressine

file = open('messages_over_time.pkl', 'rb')

messages_over_time = pkl.load(file)

file.close()

In [5]:
#visualizzo ultimo elemento dell'array di messaggi prodotti dal modello

messages_over_time[-1]

,x1,y1,vx1,vy1,q1,m1,x2,y2,vx2,vy2,q2,m2,e0,e1,dx,dy,r,epoch,loss
0,1.375472,0.137253,-0.423493,0.636396,-0.607306,5.785905,-0.324817,-2.539664,0.241439,-2.611035,0.246391,0.836671,0.002079,-0.005498,1.700289,2.676917,3.171256,199,25.336432
1,1.375472,0.137253,-0.423493,0.636396,-0.607306,5.785905,1.836016,0.545252,1.516933,2.493954,-0.804028,0.770745,0.009879,-0.060064,-0.460544,-0.407999,0.615276,199,25.336432
2,1.375472,0.137253,-0.423493,0.636396,-0.607306,5.785905,-0.495216,1.383684,0.049055,-1.245884,1.479105,0.800204,-0.003447,-0.004975,1.870688,-1.246431,2.247902,199,25.336432
3,-0.324817,-2.539664,0.241439,-2.611035,0.246391,0.836671,1.375472,0.137253,-0.423493,0.636396,-0.607306,5.785905,-0.001399,-0.023813,-1.700289,-2.676917,3.171256,199,25.336432
4,-0.324817,-2.539664,0.241439,-2.611035,0.246391,0.836671,1.836016,0.545252,1.516933,2.493954,-0.804028,0.770745,0.002454,-0.008792,-2.160833,-3.084916,3.766418,199,25.336432
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,0.493165,1.009355,2.663928,1.464371,0.583524,0.163864,0.932346,-0.275797,-0.665421,-0.685835,1.993893,0.695851,0.012113,-0.000970,-0.439181,1.285152,1.358122,199,25.336432
11996,0.493165,1.009355,2.663928,1.464371,0.583524,0.163864,1.567417,1.804571,0.422407,0.956249,0.029661,1.765544,0.007685,-0.027319,-1.074252,-0.795216,1.336558,199,25.336432
11997,1.567417,1.804571,0.422407,0.956249,0.029661,1.765544,-0.063073,-0.957318,1.110744,-0.642161,0.211620,0.581951,0.002079,-0.005498,1.630490,2.761889,3.207262,199,25.336432
11998,1.567417,1.804571,0.422407,0.956249,0.029661,1.765544,0.932346,-0.275797,-0.665421,-0.685835,1.993893,0.695851,0.002563,-0.004514,0.635071,2.080368,2.175143,199,25.336432


In [6]:
#selezioniamo la migliore componente dell'array di messaggi:
#se nel training è stato usato un modello bottleneck, le componenti tra cui scegliere saranno due...perchè si è forzato il sistema ad avere la stessa dimensionalità del problema fisico
#nel caso di regularizzation loss l1, la dimensionalità del messaggio era di 100 componenti.

import numpy as np

best_message = np.argmax([np.std(messages_over_time[-1]['e%d'%(i,)]) for i in range(2)]) 

In [7]:
#visualizzo il migliore messaggio:

messages_over_time[-1][['e%d'%(best_message,), 'dx', 'dy', 'r', 'm1', 'm2']]

,e1,dx,dy,r,m1,m2
0,-0.005498,1.700289,2.676917,3.171256,5.785905,0.836671
1,-0.060064,-0.460544,-0.407999,0.615276,5.785905,0.770745
2,-0.004975,1.870688,-1.246431,2.247902,5.785905,0.800204
3,-0.023813,-1.700289,-2.676917,3.171256,0.836671,5.785905
4,-0.008792,-2.160833,-3.084916,3.766418,0.836671,0.770745
...,...,...,...,...,...,...
11995,-0.000970,-0.439181,1.285152,1.358122,0.163864,0.695851
11996,-0.027319,-1.074252,-0.795216,1.336558,0.163864,1.765544
11997,-0.005498,1.630490,2.761889,3.207262,1.765544,0.581951
11998,-0.004514,0.635071,2.080368,2.175143,1.765544,0.695851


In [8]:

pip install pysr


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 61 kB 3.0 MB/s 
     |████████████████████████████████| 67 kB 5.0 MB/s 


In [9]:
from pysr import PySRRegressor


In [10]:
best = messages_over_time[-1][['e%d'%(best_message,), 'dx', 'dy', 'r', 'm1', 'm2']]

In [12]:
#obiettivo della symbolic regression: cercare una relazione tra le variabili "dx, dy, r, m1..." e la migliore componente del messaggio("fisicamente rappresenterebbe la forza")
#le mie variabili indipendenti saranno appunto "dx, dy, r,..."
#mentre la variabile dipendente sarà la componente migliore "e_indexnumber"

X = best[['dx', 'dy', 'r', 'm1', 'm2']]
Y = best[['e%d'%(best_message,)]]

In [ ]:
#qui si costruisce il modello per la symbolic regression:
# 1) model_selection = 'best' selezione del modello migliore in base ad una combinazione di semplicità e accuratezza
# 2) seleziona il numero di iterazioni dell'algoritmo
# 3) è possibile poi, selezionando "binary_operators" o "unary_operators", forzare il sistema a scegliere all'interno di un insieme di opratori dati
# 4) "select_k_features = somenumber" con questo comando invece chiediamo al modello di non usare tutte le variabili date (dx, dy, r, m1, m2, q1, q2) ma di scegliere le più significative per la regressione
# 5) infine si definisce la funzione di loss che dovrà quantificare la bontà dei modelli sviluppati


model = PySRRegressor(
    model_selection="best",
    niterations=300,
     binary_operators=["+", "*","-","/"],
     #unary_operators=["cos", "exp", "sin", "inv(x) = 1/x"],
     unary_operators=["square", "cube"],
     #extra_sympy_mappings={"inv": lambda x: 1/x, "inv_q": lambda x: 1/x^2, "inv_cub": lambda x: 1/x^3},
     select_k_features=3,
    loss="L1DistLoss()",
    batching=True
)

model.fit(X, Y)

/usr/local/lib/python3.7/dist-packages/pysr/sr.py:1251: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  "Note: it looks like you are running in Jupyter. "


Using features ['dy' 'r' 'm2']


In [25]:
print(model)

PySRRegressor.equations_ = [
	   pick     score                            equation      loss  complexity
	0        0.000000                       -0.0055025187  0.151785           1
	1        0.000667         cube(3.74056e-5 / (r * dy))  0.151279           6
	2  >>>>  0.002435  cube((3.74056e-5 * m2) / (r * dy))  0.150544           8
]


In [26]:
print(model.latex_table())

\usepackage{breqn}
\usepackage{booktabs}

...

\begin{table}[h]
\begin{center}
\begin{tabular}{@{}cccc@{}}
\toprule
Equation & Complexity & Loss & Score \\
\midrule
$y = -0.00550$ & $1$ & $0.152$ & $0.0$ \\
$y = \frac{5.23 \cdot 10^{-14}}{dy^{3} r^{3}}$ & $6$ & $0.151$ & $0.000667$ \\
\begin{minipage}{0.8\linewidth} \vspace{-1em} \begin{dmath*} y = \frac{5.23 \cdot 10^{-14} m_{2}^{3}}{dy^{3} r^{3}} \end{dmath*} \end{minipage} & $8$ & $0.151$ & $0.00244$ \\
\bottomrule
\end{tabular}
\end{center}
\end{table}


In [ ]:
X[["r","m1"]].shape


(12000, 2)